In [2]:
import sklearn.datasets as skd

In [3]:
categories=['alt.atheism','soc.religion.christian','comp.graphics','sci.med']

In [4]:
news_train=skd.load_files('D:\\RESEARCH PROJECT\\text classification\\20news-bydate-train',categories=categories,encoding='ISO-8859-1')

In [5]:
news_test=skd.load_files('D:\\RESEARCH PROJECT\\text classification\\20news-bydate-test',categories=categories,encoding='ISO-8859-1')

In [6]:
import pandas as pd

In [7]:
col=['text']

In [8]:
train=pd.DataFrame(news_train.data,columns=col)

In [9]:
train['categories']=news_train.target

In [11]:
train.tail(5)

,text,categories
2252,From: ib@ivan.asd.sgi.com (Ivan Bach)\nSubject...,1
2253,From: topcat!tom@tredysvr.tredydev.unisys.com ...,3
2254,From: sean@whiting.mcs.com (Sean Gum)\nSubject...,1
2255,From: dingebre@imp.sim.es.com (David Ingebrets...,1
2256,From: Nigel@dataman.demon.co.uk (Nigel Ballard...,2


In [10]:
test=pd.DataFrame(news_test.data,columns=col)

In [11]:
test['categories']=news_test.target

In [14]:
test.tail(5)

,text,categories
1497,From: dh@fncrd6.fnal.gov (don husby)\nSubject:...,2
1498,From: russ@pmafire.inel.gov (Russ Brown)\nSubj...,2
1499,From: mdw33310@uxa.cso.uiuc.edu (Michael D. Wa...,3
1500,From: fishkin@parc.xerox.com (Ken Fishkin)\nSu...,1
1501,From: vicente@cenaath.cena.dgac.fr (Martin VIC...,1


In [19]:
frames = [train,test]

In [20]:
all_data = pd.concat(frames)

In [22]:
all_data.tail(5)

,text,categories
1497,From: dh@fncrd6.fnal.gov (don husby)\nSubject:...,2
1498,From: russ@pmafire.inel.gov (Russ Brown)\nSubj...,2
1499,From: mdw33310@uxa.cso.uiuc.edu (Michael D. Wa...,3
1500,From: fishkin@parc.xerox.com (Ken Fishkin)\nSu...,1
1501,From: vicente@cenaath.cena.dgac.fr (Martin VIC...,1


In [23]:
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [24]:
import re

In [32]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [35]:
from sklearn import utils

In [39]:
import numpy as np

In [25]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(gensim.models.doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [12]:
X_train=train['text']

In [13]:
X_test=test['text']

In [28]:
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')

In [29]:
all_data = X_train + X_test

In [30]:
all_data[:2]

[TaggedDocument(words=['From:', 'dpc47852@uxa.cso.uiuc.edu', '(Daniel', 'Paul', 'Checkman)', 'Subject:', 'Re:', 'Is', 'MSG', 'sensitivity', 'superstition?', 'Article-I.D.:', 'news.C5wI4F.Dt', 'Organization:', 'University', 'of', 'Illinois', 'at', 'Urbana', 'Lines:', '22', 'bruce@Data-IO.COM', '(Bruce', 'Reynolds)', 'writes:', '>Anecedotal', 'evidence', 'is', 'worthless.', 'Even', 'doctors', 'who', 'have', 'been', 'using', 'a', 'drug', '>or', 'treatment', 'for', 'years,', 'and', 'who', 'swear', 'it', 'is', 'effective,', 'are', 'often', 'suprised', '>at', 'the', 'results', 'of', 'clinical', 'trials.', 'Whether', 'or', 'not', 'MSG', 'causes', 'describable,', '>reportable,', 'documentable', 'symptoms', 'should', 'be', 'pretty', 'simple', 'to', 'discover.', 'I', 'tend', 'to', 'disagree-', 'I', 'think', 'anecdotal', 'evidence,', 'provided', 'there', 'is', 'a', 'lot', 'of', 'it,', 'and', 'it', 'is', 'fairly', 'consistent,', 'will', 'is', 'very', 'important.', 'First,', 'it', 'points', 'to', '

# Feature Extraction---- Doc2vec

In [33]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 3759/3759 [00:00<00:00, 1202714.83it/s]


In [36]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 3759/3759 [00:00<00:00, 1575536.00it/s]


In [37]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [40]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

# Logistic Regression

In [42]:
from sklearn.linear_model import LogisticRegression

In [44]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, news_train.target)

C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

In [48]:

logreg = logreg.fit(train_vectors_dbow, news_train.target)
y_pred = logreg.predict(test_vectors_dbow)

C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [53]:
print('accuracy %s' % accuracy_score(y_pred, news_test.target))
print(classification_report(news_test.target, y_pred,target_names=categories))

accuracy 0.8115845539280959
                        precision    recall  f1-score   support

           alt.atheism       0.78      0.74      0.76       319
soc.religion.christian       0.82      0.87      0.84       389
         comp.graphics       0.84      0.81      0.82       396
               sci.med       0.80      0.81      0.81       398

              accuracy                           0.81      1502
             macro avg       0.81      0.81      0.81      1502
          weighted avg       0.81      0.81      0.81      1502



# Xgboost

In [54]:
from xgboost import XGBClassifier

In [55]:
model = XGBClassifier()
model.fit(train_vectors_dbow, news_train.target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [58]:
y_pred = model.predict(test_vectors_dbow)

In [59]:
print('accuracy %s' % accuracy_score(y_pred, news_test.target))
print(classification_report(news_test.target, y_pred,target_names=categories))

accuracy 0.7649800266311585
                        precision    recall  f1-score   support

           alt.atheism       0.68      0.65      0.66       319
soc.religion.christian       0.81      0.84      0.83       389
         comp.graphics       0.83      0.75      0.79       396
               sci.med       0.73      0.80      0.76       398

              accuracy                           0.76      1502
             macro avg       0.76      0.76      0.76      1502
          weighted avg       0.77      0.76      0.76      1502



# removing noise to check if accuracy can be improved

In [62]:
all_data = all_data[pd.notnull(all_data['text'])]

TypeError: list indices must be integers or slices, not str

In [66]:
train['text'].apply(lambda x: len(x.split(' '))).sum()

752658

In [14]:
###cleaning the text
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abishek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import re

In [17]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    ##text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [18]:
train['post'] = train['text'].apply(clean_text)

In [19]:
train['post'].apply(lambda x: len(x.split(' '))).sum()

374715

In [68]:
train.head(5)

,text,categories,post
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...,2,dpc47852 uxacsouiucedu daniel paul checkman su...
1,From: yoo@engr.ucf.edu (Hoi Yoo)\nSubject: loo...,1,yoo engrucfedu hoi yoo subject looking usa map...
2,From: fernandeza@merrimack.edu\nSubject: Re: T...,3,fernandeza merrimackedusubject arrogance chris...
3,From: mcelwre@cnsvax.uwec.edu\nSubject: NATURA...,2,mcelwre cnsvaxuwecedusubject natural anticance...
4,From: mathew <mathew@mantis.co.uk>\nSubject: R...,0,mathew mathew mantiscouksubject inimitable rus...


In [70]:
test['text'].apply(lambda x: len(x.split(' '))).sum()

570522

In [21]:
test['post'] = test['text'].apply(clean_text)

In [22]:
test['post'].apply(lambda x: len(x.split(' '))).sum()

281893

In [74]:
X_train1=train['post']
X_test1=test['post']

In [77]:
X_train1 = label_sentences(X_train1, 'Train')
X_test1 = label_sentences(X_test1, 'Test')

In [78]:
all_data1 = X_train1 + X_test1

In [79]:
all_data[:2]

[TaggedDocument(words=['From:', 'dpc47852@uxa.cso.uiuc.edu', '(Daniel', 'Paul', 'Checkman)', 'Subject:', 'Re:', 'Is', 'MSG', 'sensitivity', 'superstition?', 'Article-I.D.:', 'news.C5wI4F.Dt', 'Organization:', 'University', 'of', 'Illinois', 'at', 'Urbana', 'Lines:', '22', 'bruce@Data-IO.COM', '(Bruce', 'Reynolds)', 'writes:', '>Anecedotal', 'evidence', 'is', 'worthless.', 'Even', 'doctors', 'who', 'have', 'been', 'using', 'a', 'drug', '>or', 'treatment', 'for', 'years,', 'and', 'who', 'swear', 'it', 'is', 'effective,', 'are', 'often', 'suprised', '>at', 'the', 'results', 'of', 'clinical', 'trials.', 'Whether', 'or', 'not', 'MSG', 'causes', 'describable,', '>reportable,', 'documentable', 'symptoms', 'should', 'be', 'pretty', 'simple', 'to', 'discover.', 'I', 'tend', 'to', 'disagree-', 'I', 'think', 'anecdotal', 'evidence,', 'provided', 'there', 'is', 'a', 'lot', 'of', 'it,', 'and', 'it', 'is', 'fairly', 'consistent,', 'will', 'is', 'very', 'important.', 'First,', 'it', 'points', 'to', '

In [80]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data1)])

100%|██████████| 3759/3759 [00:00<00:00, 1255785.64it/s]


In [81]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data1)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 3759/3759 [00:00<00:00, 1883678.46it/s]


In [82]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train1), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test1), 300, 'Test')

In [83]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, news_train.target)

C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [84]:
logreg = logreg.fit(train_vectors_dbow, news_train.target)
y_pred = logreg.predict(test_vectors_dbow)

C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\abishek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [85]:
print('accuracy %s' % accuracy_score(y_pred, news_test.target))
print(classification_report(news_test.target, y_pred,target_names=categories))

accuracy 0.8641810918774967
                        precision    recall  f1-score   support

           alt.atheism       0.84      0.80      0.82       319
soc.religion.christian       0.86      0.93      0.89       389
         comp.graphics       0.91      0.85      0.88       396
               sci.med       0.84      0.87      0.85       398

              accuracy                           0.86      1502
             macro avg       0.86      0.86      0.86      1502
          weighted avg       0.87      0.86      0.86      1502



In [86]:
model = XGBClassifier()
model.fit(train_vectors_dbow, news_train.target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [87]:
y_pred = model.predict(test_vectors_dbow)

In [88]:
print('accuracy %s' % accuracy_score(y_pred, news_test.target))
print(classification_report(news_test.target, y_pred,target_names=categories))

accuracy 0.8468708388814914
                        precision    recall  f1-score   support

           alt.atheism       0.84      0.75      0.79       319
soc.religion.christian       0.89      0.90      0.90       389
         comp.graphics       0.88      0.83      0.85       396
               sci.med       0.79      0.89      0.84       398

              accuracy                           0.85      1502
             macro avg       0.85      0.84      0.84      1502
          weighted avg       0.85      0.85      0.85      1502



# Feature extraction-   bag of words==> TF-IDF

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [24]:
x_train=train['post']


In [25]:
nb.fit(x_train,news_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
x_test=test['post']

In [29]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print('accuracy %s' % accuracy_score(y_pred, news_test.target))
print(classification_report(news_test.target, y_pred,target_names=categories))

accuracy 0.8708388814913449
                        precision    recall  f1-score   support

           alt.atheism       0.99      0.65      0.78       319
soc.religion.christian       0.96      0.94      0.95       389
         comp.graphics       0.95      0.87      0.91       396
               sci.med       0.71      0.98      0.83       398

              accuracy                           0.87      1502
             macro avg       0.90      0.86      0.87      1502
          weighted avg       0.90      0.87      0.87      1502

Wall time: 429 ms


# trying with xgboost

In [98]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('model', XGBClassifier()),
              ])

In [99]:
nb.fit(x_train,news_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing

In [100]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print('accuracy %s' % accuracy_score(y_pred, news_test.target))
print(classification_report(news_test.target, y_pred,target_names=categories))

accuracy 0.8149134487350199
                        precision    recall  f1-score   support

           alt.atheism       0.92      0.69      0.79       319
soc.religion.christian       0.74      0.93      0.82       389
         comp.graphics       0.79      0.79      0.79       396
               sci.med       0.86      0.83      0.85       398

              accuracy                           0.81      1502
             macro avg       0.83      0.81      0.81      1502
          weighted avg       0.83      0.81      0.81      1502

Wall time: 467 ms


In [30]:
def Convert(string): 
    li = list(string.split(",")) 
    return li

In [31]:
stringinput=input("enter news")


enter newsThe graphics card improved a lot in these days


In [32]:
news=Convert(stringinput)

In [33]:
news=pd.DataFrame(news,columns=['news'])

In [34]:
news_pred=news['news']

In [35]:
y_pred = nb.predict(news_pred)

In [36]:
y_pred

array([1])

0-alt.aetheism
1-comp.graphics
2-sci.med
3-soc.religion.christian